In [ ]:
# !conda install -n arcgis -q -y sqlalchemy psycopg2 python-slugify geopandas geoalchemy2

In [ ]:
# import arcgis
import logging
import geopandas as gpd
import pandas as pd

from datetime import datetime
from pathlib import Path
from slugify import slugify
from sqlalchemy import create_engine, text, \
    Table, MetaData, Column, \
    Integer, String, BigInteger, Boolean, \
    Float, DateTime, Numeric, \
    UniqueConstraint, CheckConstraint, ForeignKeyConstraint
from geoalchemy2 import Geometry

logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Coletar os dados brutos e salvar em Parquet

## Informações de conexão de origem

In [ ]:
with open("./dsn/source.txt") as r:
    src_engine = create_engine(r.read())

## Informações de template do Parquet

In [ ]:
def save_parquet(df, prefix, schema, table, suffix="", output_path="./data/"):

    if suffix and not suffix.startswith("_"):
        suffix = "_" + str(suffix)
    
    _file = Path(output_path) / prefix / f"{schema}_{table}{suffix}.parquet"
    
    if not _file.exists():
        _file.parent.mkdir(parents=True, exist_ok=True)
    
    df.to_parquet(_file, index=True)

    return  _file

## Informações de tabela de origem

In [ ]:
# Iniciais
src_table_prefix = "resultados_analiticos"
tables = ["agua", "concentrado_de_bateia", "mineral_minerio", "outros", "rocha", "sedimento_de_corrente", "solo", "vegetacao"]
table_item = tables[5]

# Argumentos para rodar o pipeline
src_table_name = f"{src_table_prefix}_{table_item}"
src_schema = "sde"
src_primary_key = "objectid"

# Coordenadas
src_x_field = "longitude"
src_y_field = "latitude"
src_srid = 4674

src_table_name

### Ler tabela de origem e salvar em Parquet

In [ ]:
# Puxar os dados e converter em dataframe
try: 
    df = (
        pd.read_sql_table(src_table_name, src_engine, schema=src_schema, index_col=src_primary_key)
            # Sanitizar colunas, para não inserir valores fora de padrão na DDL do banco (acentos, caracteres especiais, etc)
            .rename(columns=lambda col: slugify(col, separator="_"))
    )
    
except Exception as e:
    logging.error(e)

assert not df.empty, "Esta tabela está vazia. Abortando..."
assert df.index.is_unique, f"Esta tabela não possui identifcador único de registro no campo <{primary_key}>"

out_raw_file = save_parquet(df, datetime.now().strftime("%Y/%m/%d"), src_schema, src_table_name, suffix="raw")

# Info
df.info()

# Tratamento dos dados brutos e salvar em PostgreSQL

## Informações de conexão de destino

In [ ]:
with open("./dsn/destiny.txt") as r:
    dst_engine = create_engine(r.read(), plugins=["geoalchemy2"])

metadata = MetaData(schema="public")

## Tabela de amostras (survey)

### Análise de colunas

In [ ]:
# Construção das colunas
pk_column = Column(src_primary_key, BigInteger(), primary_key=True, autoincrement=False)
geom_column = Column('geometry', Geometry('POINT', srid=src_srid, dimension=2), nullable=False)

fixed_columns = (
    Column("projeto_amostragem", String(255), nullable=False),
    Column("projeto_publicacao", String(255), nullable=False),
    Column("centro_de_custo", String(10), nullable=False),
    Column("classe", String(50), nullable=False),
    Column("numero_de_campo", String(20), nullable=False),
    Column("numero_de_laboratorio", String(8), nullable=False),
    Column("duplicata", Boolean(), nullable=False, default=False, server_default="0"),
    Column(src_x_field, Float(), nullable=False),
    Column(src_y_field, Float(), nullable=False),
    Column("laboratorio", String(255), nullable=False),
    Column("job", String(30), nullable=True),
    Column("data_de_analise", DateTime(), nullable=True),
    Column("observacao", String(1024), nullable=True),    
)

# Colunas extras
if table_item in ("mineral_minerio", "rocha"):
    extra_columns = (
        Column("classificacao_petrografica", String(255), nullable=True),
        Column("unidade_litoestratigrafica", String(255), nullable=True),
    ) 
else:
    extra_columns = ()


# Juntando as colunas comuns ao dataframe
survey_columns = fixed_columns[:-2] + extra_columns + fixed_columns[-2:]
survey_column_names = tuple([col.name for col in survey_columns])

# Verificação se a coluna existe no DataFrame
for col in survey_column_names:
    assert col in df.columns, f"The fixed column <{col}> is not in table columns."

# Adicionar pk e geometria na tupla final de colunas
survey_columns = (
    pk_column,
    *survey_columns, 
    geom_column
)
# survey_columns

### Criação do objeto SQL Table

In [ ]:
survey_table_name = f"{table_item}_amostras"

survey_tbl = Table(
    survey_table_name,
    metadata,
    *survey_columns,
    extend_existing=True  # !!!!
)

# survey_tbl

### Extração de dados de amostras

In [ ]:
# GeoPandas
survey_df = gpd.GeoDataFrame(
    df.filter(survey_column_names)
        .apply(lambda col: col.replace("", None))
        .rename(columns=lambda col: slugify(col, separator="_"))    
        .assign(
            # forçar duplicata como booleano
            duplicata=lambda df: df.duplicata.fillna("não").str.match("(sim|1)", case=False)
        ),
    geometry=gpd.points_from_xy(
        df[src_x_field], 
        df[src_y_field], 
        crs=src_srid
    )
)

# ObjectID tem que ser único e não pode ter geometria nula ou vazia
assert survey_df.index.is_unique, f"ObjectID precisa ser único: {survey_df[survey_df.index.duplicated()].index.tolist()}"
assert not (survey_df.geometry.isna().all() and survey_df.geometry.is_empty.all()), "A tabela não pode ter geometria nula ou vazia"

# Gravar em Parquet
# out_survey_file = save_parquet(survey_df, "silver", src_schema, src_table_name, suffix="survey")

# Info
survey_df.info()

## Tabelas de análises (assays)

### Análise de colunas

In [ ]:
# Colunas a excluir do processo
assay_excluded_columns = ("globalid", "lote", "ra", "metodo", "created_user", "created_date", "last_edited_user", "last_edited_date")

# Colunas a serem pivotadas
assay_columns = [col for col in df.columns if col not in (survey_column_names + assay_excluded_columns)]

# Colunas para a tabela de resultados analíticos
assay_columns_fixed = (
    Column("abertura", String(100), nullable=False),
    Column("leitura", String(100), nullable=False)
)

for col in assay_columns_fixed:
    assert col.name in df.columns, f"The fixed column <{col}> is not in assay columns."

assay_sample_column = Column("amostra", BigInteger(), nullable=False)
assay_analyte_column = Column("analito", String(10), nullable=False)
assay_unit_column = Column("unidade", String(6), nullable=False)
assay_qualif_column = Column("qualificador", String(1), nullable=True)
assay_value_column =  Column("valor", Numeric(), nullable=False)


assay_pivoted_columns = (
    Column("id", BigInteger(), primary_key=True, autoincrement=False),
    assay_sample_column,
    *assay_columns_fixed,
    assay_analyte_column,
    assay_unit_column,
    assay_qualif_column,
    assay_value_column
)

# assay_pivoted_columns

### Criação do objeto SQL Table

In [ ]:
assay_table_name = f"{table_item}_analises_quimicas"

assay_tbl = Table(
    assay_table_name,
    metadata,
    *assay_pivoted_columns,
    UniqueConstraint("amostra", "abertura", "leitura", "analito", "unidade", name=f"{assay_table_name}_uniq"),
    CheckConstraint("qualificador IN ('<', '>')", name=f"{assay_table_name}_qualif_chk"),
    ForeignKeyConstraint(
        [assay_sample_column.name],
        [f"{survey_table_name}.{src_primary_key}"],
        onupdate="RESTRICT",
        ondelete="CASCADE",
    ),
    extend_existing=True  # !!!!
)

# assay_tbl

### Extração de dados de análises

In [ ]:
# Data cleaning constants
spaces_regex = r"\s+"
missing_values = ["ND", "", "N.A."] + [" ", "#N/A", "#N/A N/A", "#NA", "-1.#IND", "-1.#QNAN", "-NaN", "-nan", "1.#IND", "1.#QNAN", "<NA>", "N/A", "NA", "NULL", "NaN", "None", "n/a", "nan", "null"]
normalize_values = {
    ',': ".",
    "-": "<",
    "<.": "<0.",
}

# Pipes
def handle_missing(series, extra_missing_values=[]):
    return (
        series.str.replace(spaces_regex, "", regex=True)
            .replace(missing_values + extra_missing_values, None) 
    )

def handle_normalized(series, replaces={}):
    for key, value in replaces.items():
        series = series.str.replace(key, value)
    return series
    
# Primeiras limpezas
index_names = [col.name for col in (assay_sample_column, *assay_columns_fixed, assay_analyte_column)]
value_name = f"{assay_value_column.name}_{assay_qualif_column.name}"

assay_df = (
    df.filter(assay_columns)
        .apply(lambda col: col.replace("", None))
        .rename(columns=lambda col: slugify(col, separator="_"))
        # Traz o objectid para o index do dataframe
        .set_index([col.name for col in assay_columns_fixed], append=True)
        # De-pivot
        .stack()
        # Ajusta nomes
        .rename_axis(index_names)
        .rename(value_name)
        # handle missing data on values
        .pipe(handle_missing)
        .dropna()
        # normalize values  qualificators
        .pipe(handle_normalized, normalize_values)
)

# Tratamento de index
_analyte_name, _unit_name = assay_analyte_column.name, assay_unit_column.name

assay_df = (
    assay_df
        .to_frame()
        .join(
            pd.DataFrame(
                assay_df.index.get_level_values(_analyte_name).str.rsplit("_", n=1, expand=True).to_list(), 
                    index=assay_df.index, 
                    columns=[_analyte_name, _unit_name]
                )
        )
        .reset_index(level=_analyte_name, drop=True)
        .set_index([_analyte_name, _unit_name], append=True)
)

# ObjectID tem que ser único
assert survey_df.index.is_unique

# Decomposição de valores para valor, qualificador
# Valores precisam atender a padrão de valores
values_match = assay_df.squeeze().astype(str).str.match(r"^[<>]?\d+\.?\d*$")
assert values_match.all(), f"Alguns valores ({assay_df[~values_match].shape[0]}) não coincidiram com a expressão regular:\n {assay_df[~values_match].head()}"

assay_df = (
    assay_df.join(
        assay_df[value_name].str.extract(r'(^[<>]?)(\d+\.?\d*)$').rename(columns=dict(enumerate(value_name.split("_")[::-1])))
    )
    .assign(
        valor = lambda df: pd.to_numeric(df.valor, errors="raise"),
        qualificador = lambda df: df.qualificador.pipe(handle_missing).astype("category")
    )
    .drop(
        value_name,
        axis="columns"
    )    
)

# # Gravar em Parquet
# out_assay_file = save_parquet(assay_df, "silver", src_schema, src_table_name, suffix="assay")

# Info
assay_df.info()

### Análises não validadas

In [ ]:
assay_df[~values_match]

# Mandar para o banco de dados de destino

### Criar estruturas de SQL usando o SQLALchemy

In [ ]:
# Destrói as duas tabelas
metadata.drop_all(dst_engine)
metadata.create_all(dst_engine)

### Gravar do Parquet para o banco de dados

In [ ]:
# Gravar o parquet no banco de dados
with dst_engine.connect() as conn:
    with conn.begin():
        # 'to_postgis' não funciona com method. Ver como fazer isso com 'to_sql' tradicional
        logging.info("Gravando amostras...")
        (survey_df
            .to_postgis(
                survey_table_name, 
                conn, 
                if_exists='append', 
                schema="public", 
                index=True, 
                chunksize=5000
            )
        )

        logging.info("Gravando análises...")
        (assay_df
            .reset_index()
            .rename_axis(assay_pivoted_columns[0].name)
            .to_sql(
                assay_table_name, 
                conn, 
                if_exists='append', 
                schema="public", 
                index=True, 
                chunksize=10000, 
                # method="multi" # https://pandas.pydata.org/docs/user_guide/io.html#io-sql-method
            )
        )
        
        conn.commit()
        
    logging.info("Reindexando...")
    for tbl in [survey_table_name, assay_table_name]:
        conn.execute(text(f"REINDEX TABLE {tbl};"))
        
    logging.info("Finalizado!")